# Record Rename

Get rid of the `record` dimension in the RRTMGP flux netCDF files and replace it with `forcing` (working title) since the dimension represents different forcing scenarios.

The only non-standard library is `xarray`, which the user must install in their own Python environment. `PIPPATH` is the assumed location.

In [ ]:
import os, sys

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
sys.path.append(PIPPATH)

import xarray as xa

## Paths

The user should provide an existing RRTMGP (or LBLRTM) netCDF with fluxes and heating rates in it. This is different from the netCDF from which RRTMGP reads absorption coefficients.

A filename for the data with a `forcing` (working title) dimension should also be provided, but it is not necessary. If `overwrite` is set to `True`, the input netCDF will have its data and attributes replaced. Otherwise, the new dataset will be written to `outNC`.

In [ ]:
inNC = 'temp-lbl.nc'
assert os.path.exists(inNC), 'Could not find {}'.format(inNC)

overwrite = True

outNC = str(inNC) if overwrite else 'lblrtm-lw-flux-inputs-outputs-forcing.nc'

## Dimension Modification

In this cell, we will open up `inNC` and iterate over all of its variables, extracting their information and modifying some of it. If a given variable has a `record` dimension, we will rename it to `forcing` (working title). If not, the variable is retained as is. Then we write to `outNC` the modified and unmodified variables with their attributes, along with unmodified global attributes from `inNC.

In [ ]:
outDS = xa.Dataset()

with xa.open_dataset(inNC) as inObj:
    # save global attributes for later -- will stuff into buffer, unedited
    globalAtt = inObj.attrs

    # write buffer netCDF, complete with global attributes
    ncVars = list(inObj.keys())

    for ncVar in ncVars: 
        ncDat = inObj[ncVar]

        if 'record' in ncDat.dims:
            # which dimension corresponds to `record`?
            dims = list(ncDat.dims)
            iRec = dims.index('record')
            dims[iRec] = 'forcing'

            # save variable with new dimensions
            outDS[ncVar] = xa.DataArray(ncDat, dims=dims)
        else:
            # retain any variables without a record dimension
            outDS[ncVar] = xa.DataArray(ncDat)
        # endif record
    # end ncVar loop
# endwith

# stuff the global attributes into the new dataset
for att in globalAtt: outDS.attrs[att] = globalAtt[att]
outDS.to_netcdf(outNC, mode='w')
print('Completed {}'.format(outNC))